In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import torch, time, sys
import numpy as np
import scipy.integrate
solve_ivp = scipy.integrate.solve_ivp

In [ ]:
import os

cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))

In [ ]:
import pickle

with open('expert_rollout.pickle', 'rb') as dbfile:
    expert_rollout = pickle.load(dbfile)
with open('airl_rollout.pickle', 'rb') as dbfile:
    baseln_rollout = pickle.load(dbfile)
# with open('hnn_rollout.pickle', 'rb') as dbfile:  # TODO: Uncomment after running improved algo
#     improv_rollout = pickle.load(dbfile)


In [ ]:
obs_expert = np.array([traj.obs for traj in expert_rollout])[4:,...]
obs_baseln = np.array([traj.obs for traj in baseln_rollout])[4:,...]
# obs_improv = np.array([traj.obs for traj in improv_rollout])  # TODO: Uncomment after running improved algo

In [ ]:
obs_expert[..., 0].shape

In [ ]:
def obs_to_angles(observations: np.ndarray) -> np.ndarray:
    """Given a (N, 2) array of cosines and sines,
    return a (N,) array of angles (in radians)
    """
    cosines, sines = observations[..., 0], observations[..., 1]
    angles = np.arccos(cosines)
    return np.where(sines < 0, -angles, angles)


In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, dpi=100, figsize=(15., 10.), facecolor='w')
colors = {'expert': 'black', 'baseline': 'blue', 'improved': 'red'}
fig.suptitle('Angles as functions of steps')
for idx in range(2):
    # axes[idx].set_title(fr'$\theta_{idx+1}$ as a function of steps')
    for key, obs in {'expert': obs_expert,
                     'baseline': obs_baseln,
                     # 'improved': obs_improv,  # TODO: Uncomment after obtaining obs_improv
                     }.items():
        thetas = obs_to_angles(obs[..., (2*idx):(2*idx+2)])
        # https://en.wikipedia.org/wiki/Circular_mean
        # See above for the rationale for computing angle mean as below:
        theta_mean = obs_to_angles(np.mean(obs[..., (2*idx):(2*idx+2)], axis=0))
        assert thetas.ndim == 2 and theta_mean.ndim == 1
        assert (thetas.shape[1],) == theta_mean.shape
        axes[idx].plot(thetas.T, c=colors[key], alpha=0.1)
        axes[idx].plot(theta_mean, c=colors[key], label=key)
    axes[idx].set_ylim(-np.pi, np.pi)
    axes[idx].set_yticks(
        [-np.pi, -np.pi/2, 0., np.pi/2, np.pi],
        labels=[r'$-\pi$', r'$-\pi/2$', r'$0$', r'$\pi/2$', r'$\pi$'],
    )
    axes[idx].set_ylabel(fr'$\theta_{idx+1}$ (radians)')
    axes[idx].set_xlim(0, 512)
    axes[idx].set_xlabel('steps')
    axes[idx].legend()
plt.tight_layout()
plt.grid()
plt.show()

In [ ]:
rew_expert = np.array([traj.rews for traj in expert_rollout])
rew_baseln = np.array([traj.rews for traj in baseln_rollout])

In [ ]:
plt.plot(rew_expert.T, color='black', alpha=0.1)
plt.plot(np.mean(rew_expert, axis=0), color='black', label='expert')
plt.plot(rew_baseln.T, color='blue', alpha=0.1)
plt.plot(np.mean(rew_baseln, axis=0), color='blue', label='baseline')
plt.xlim(0, 512)
plt.ylim(-1, 0)
plt.xlabel('steps')
plt.ylabel('reward')
plt.title('Expected immediate reward as a function of steps')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
# obs_expert_0 =np.transpose(np.array(expert_rollout[0].obs))
# obs_expert_1 =np.transpose(np.array(expert_rollout[1].obs))

# lw = 3 #linewidth
# fs=9
# ts=15
# tpad = 7
# ls=12

# fig = plt.figure(figsize=[15,4], dpi=100)
# plt.subplot(1,3,1)
# plt.title('Trajectories', fontsize=ts, pad=tpad)
# colors = ['orange', 'purple']

# plt.plot(obs_expert_0[1], obs_expert_0[2], '-', c=colors[0], label='True path, roll out {}'.format(0), linewidth=2)
    
# plt.plot(obs_expert_1[1], obs_expert_1[2], '--', c=colors[1], label='True path, roll out {}'.format(1), linewidth=2)

# plt.axis('equal')
# plt.xlabel('$x$', fontsize=ls) ; plt.ylabel('$y$', fontsize=ls)
# plt.legend(fontsize=fs)
# """
# plt.subplot(1,3,2)
# real_pe, real_ke, real_etot = potential_energy(orbit), kinetic_energy(orbit), total_energy(orbit)
# plt.title('Ground truth energy', fontsize=ts, pad=tpad)
# plt.xlabel('Time')
# plt.plot(settings['t_eval'], real_pe, 'g:', label='Potential', linewidth=lw)
# plt.plot(settings['t_eval'], real_ke, 'c-.', label='Kinetic', linewidth=lw)
# plt.plot(settings['t_eval'], real_etot, 'k-', label='Total', linewidth=lw)
# plt.legend(fontsize=fs)
# plt.xlim(*settings['t_span'])
# ymin = np.min([real_pe.min(), real_ke.min(), real_etot.min()])
# ymax = np.max([real_pe.max(), real_ke.max(), real_etot.max()])
# plt.ylim(ymin, ymax)

# plt.subplot(1,3,3)
# plt.title('Baseline NN energy', fontsize=ts, pad=tpad)
# plt.xlabel('Time')
# plt.plot(settings['t_eval'], potential_energy(base_orbit), 'g:', label='Potential', linewidth=lw)
# plt.plot(settings['t_eval'], kinetic_energy(base_orbit), 'c-.', label='Kinetic', linewidth=lw)
# plt.plot(settings['t_eval'], total_energy(base_orbit), 'k-', label='Total', linewidth=lw)
# plt.legend(fontsize=fs)
# plt.xlim(*settings['t_span'])
# plt.ylim(ymin, ymax)
# """
# plt.tight_layout() ; plt.show()
# #fig.savefig('{}/orbits-base-example.{}'.format(args.fig_dir, FORMAT))